In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import sqlalchemy

In [2]:
engine = sqlalchemy.create_engine('sqlite:///data_db.db')

In [3]:
with open("media/20210601-1834_prototype1_wifi_scan-01.csv", "r") as f:
    df = pd.read_csv(f)

FileNotFoundError: [Errno 2] No such file or directory: 'media/20210601-1834_prototype1_wifi_scan-01.csv'

In [5]:
print(df)

               BSSID       First time seen        Last time seen  channel  \
0  A6:08:F5:68:D5:17   2021-06-01 18:34:51   2021-06-01 18:34:55        1   
1  A4:08:F5:68:D3:16   2021-06-01 18:34:51   2021-06-01 18:34:55        1   
2  7A:97:41:0F:33:D7   2021-06-01 18:34:54   2021-06-01 18:34:55        6   
3  70:97:41:0F:33:D3   2021-06-01 18:34:54   2021-06-01 18:34:55        6   
4  7A:97:41:0F:33:D4   2021-06-01 18:34:54   2021-06-01 18:34:54        6   
5        Station MAC       First time seen        Last time seen    Power   

        Speed  Privacy          Cipher  Authentication   Power   # beacons  \
0          54      OPN                                   -57.0         5.0   
1          54     WPA2            CCMP             PSK   -56.0         5.0   
2          54     WPA2            CCMP             PSK   -26.0         2.0   
3          54     WPA2            CCMP             PSK   -27.0         2.0   
4          54      OPN                                   -27.0        

In [6]:
mac_list = [mac for mac in df.iloc[:-1,0]]
print(mac_list)


['A6:08:F5:68:D5:17', 'A4:08:F5:68:D3:16', '7A:97:41:0F:33:D7', '70:97:41:0F:33:D3', '7A:97:41:0F:33:D4']


In [7]:
df.to_sql('test', engine, if_exists='append', index=False)

In [9]:
macs = pd.read_sql('test', engine)
macs = macs['BSSID']
macs = [mac for mac in macs if mac != "Station MAC"] # csv/df needs cleaning, unpredictable second header row
print(macs)

['A6:08:F5:68:D5:17', 'A4:08:F5:68:D3:16', '7A:97:41:0F:33:D7', '70:97:41:0F:33:D3', '7A:97:41:0F:33:D4', 'A6:08:F5:68:D5:17', 'A4:08:F5:68:D3:16', '7A:97:41:0F:33:D7', '70:97:41:0F:33:D3', '7A:97:41:0F:33:D4']


In [16]:
mac_prefixes = [mac[:8].replace(":", "") for mac in macs] # organisational identifier
mac_suffixes = [mac[9:].replace(":", "") for mac in macs] # personal identifier
print("prefixes")
print(mac_prefixes)
print("suffixes")
print(mac_suffixes)

prefixes
['A608F5', 'A408F5', '7A9741', '709741', '7A9741', 'A608F5', 'A408F5', '7A9741', '709741', '7A9741']
suffixes
['68D517', '68D316', '0F33D7', '0F33D3', '0F33D4', '68D517', '68D316', '0F33D7', '0F33D3', '0F33D4']


https://maclookup.app/search/result?mac=A408F5

In [17]:
import requests

In [47]:
def mac_lookup(macaddr):
    headers = {'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'}
    r = requests.get(f'https://maclookup.app/search/result?mac={macaddr}', headers = headers)
    c = r.content
    soup = BeautifulSoup(c, "html.parser")
    try:
        org = soup.select_one("h2").get_text()
    except:
        org="Unknown"
    return org

In [48]:
mac_lookup("A408F5")

'Sagemcom Broadband SAS'

In [46]:
devices = [mac_lookup(prefix) for prefix in mac_prefixes]
print(devices)

['Unknown', 'Sagemcom Broadband SAS', 'Unknown', 'Arcadyan Corporation', 'Unknown', 'Unknown', 'Sagemcom Broadband SAS', 'Unknown', 'Arcadyan Corporation', 'Unknown']
